In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from sklearn.neighbors import NearestNeighbors

# Functions

## Data Cleaning

In [ ]:
def display_countplot(df, x, title=None, filename=None, x_ticks=None):
    """Function to display the countplot for the specified column of the df dataframe
    Args:
        1) df - a dataframe that contains the data required for visualization
        2) x - the name of the dataframe column on which the countplot is built
        3) title - the name of the graph (countplot)
        4) filename - the path where the file will be saved (with the file name, the file extension is not required) 
        5) x_ticks - ticks on the OX axis of the graph
    """
    plt.figure(figsize=(10, 5))
    sns.countplot(data=df, x=x)
    plt.title(title)
    if x_ticks:
        plt.xticks(x_ticks)
    if filename:
        plt.savefig(f'{filename}.png', bbox_inches='tight')
    plt.show()

# Data Cleaning and Preprocessing

In [ ]:
columns = ['pregnancies', 'glucose_concentration', 'blood_pressure', 'triceps_skin_thickness', 'serum_insulin', 'bmi',
           'diabetes_pedigree_function', 'age', 'has_diabetes']
df = pd.read_csv('data/pima-indians-diabetes.csv', names=columns)
df.head()

In [ ]:
df.info()

In [ ]:
len(df[df['pregnancies'] < 0])

### Explore unique values of each column

In [ ]:
print(f'Unique values of each column of dataset')
for column in df.columns:
    print(f"{column}: {df[column].unique()}")

#### Explore the 'pregnancies' column

In [ ]:
df['pregnancies'].unique()

In [ ]:
display_countplot(df=df, x='pregnancies', title='Number of times pregnant', filename='graphs/pregnancies_countplot')

#### Explore the 'glucose_concentration' column

In [ ]:
glucose_arr = df['glucose_concentration'].unique()
glucose_arr

In [ ]:
len(df[df['glucose_concentration']==0])

#### Explore the 'blood_pressure' column

In [ ]:
blood_pressure_arr = df['blood_pressure'].unique()
blood_pressure_arr.sort()
blood_pressure_arr

In [ ]:
len(df[df['blood_pressure']==0])

#### Explore the 'triceps_skin_thickness' column

In [ ]:
triceps_skin_thickness = df['triceps_skin_thickness'].unique()
triceps_skin_thickness.sort()
triceps_skin_thickness

In [ ]:
len(df[df['triceps_skin_thickness']==0])

##### Solution = Imputation

Imputation: If the feature is important and you decide to retain it, you can consider imputing the missing values using techniques such as mean imputation, median imputation, or more advanced methods like k-nearest neighbors imputation. Imputation can help you retain the valuable information while filling in the gaps.

#### Explore the 'serum_insulin' column

In [ ]:
serum_insulin = df['serum_insulin'].unique()
serum_insulin.sort()
serum_insulin

In [ ]:
len(df[df['serum_insulin']==0])

#### Explore the 'bmi' column

In [ ]:
bmi = df['bmi'].unique()
bmi.sort()
bmi

In [ ]:
len(df[df['bmi']==0])

#### Explore the 'diabetes_pedigree_function' column

In [ ]:
diabetes_pedigree_func_arr = df['diabetes_pedigree_function'].unique()
diabetes_pedigree_func_arr.sort()
diabetes_pedigree_func_arr

#### Explore the 'age' column

In [ ]:
age_arr = df['age'].unique()
age_arr.sort()
age_arr

#### Explore the 'has_diabetes' column

In [ ]:
df['has_diabetes'].unique()

### Perform data cleaning

In [ ]:
len(df[(df['glucose_concentration'] == 0) | (df['blood_pressure'] == 0) | (df['bmi'] == 0)])

In [ ]:
df = df[(df['glucose_concentration'] != 0) & (df['blood_pressure'] != 0) & (df['bmi'] != 0)]
df.info()

In [ ]:
len(df[df['triceps_skin_thickness'] == 0])

#### Using K-Nearest Neighbors (KNN) Imputation for the 'triceps_skin_thickness' column

##### View the correlation matrix

In [ ]:
# Construct a correlation matrix between the 'triceps_skin_thickness' column and other characteristics
correlation_matrix = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix Heatmap')
plt.savefig('graphs/correlation_matrix.png', bbox_inches='tight')
plt.show()

##### Perform KNN imputation

In [ ]:
df[df['triceps_skin_thickness'] == 0][:5]

In [ ]:
# Subset of relevant columns for KNN imputation
subset_columns = ['triceps_skin_thickness', 'serum_insulin', 'bmi']

# Separate data with missing 'triceps_skin_thickness'
missing_data = df[df['triceps_skin_thickness'] == 0][subset_columns]
known_data = df[df['triceps_skin_thickness'] != 0][subset_columns]

# Reset the index of both DataFrames
missing_data.reset_index(drop=True, inplace=True)
known_data.reset_index(drop=True, inplace=True)

# Fit a NearestNeighbors model
neighbors_number = 5
neighbors_model = NearestNeighbors(n_neighbors=neighbors_number)
neighbors_model.fit(known_data.drop('triceps_skin_thickness', axis=1).values)

# Impute missing values using KNN
for index, row in missing_data.iterrows():
    query_point = row.drop('triceps_skin_thickness')
    distances, indices = neighbors_model.kneighbors([query_point])
    avg_triceps_thickness = np.mean([known_data.loc[idx, 'triceps_skin_thickness'] for idx in indices[0]])
    df.loc[df.index == index, 'triceps_skin_thickness'] = avg_triceps_thickness

len(df[df['triceps_skin_thickness'] == 0])